In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf

In [ ]:
# tf.config.set_soft_device_placement(True)

In [ ]:

# tf.debugging.set_log_device_placement(True)

In [ ]:
# try:
#     # TPU detection. No parameters necessary if TPU_NAME environment variable is
#     # set: this is always the case on Kaggle.
#     tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
#     print('Running on TPU ', tpu.master())
# except ValueError:
#     tpu = None

    

In [ ]:
tf

In [ ]:
# tf.config.experimental_connect_to_cluster(tpu)

# # Initialize TPU system
# tf.tpu.experimental.initialize_tpu_system(tpu)

# # Create TPUStrategy
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
!pip install opencv-python-headless

In [ ]:
!pip install imageio

In [ ]:
!pip install gdown

In [ ]:
import pandas as pd 
import numpy as np 

import cv2
import imageio 
import os
import matplotlib.pyplot as plt
%matplotlib inline
from typing import List

In [ ]:
import gdown
url = 'https://drive.google.com/uc?id=1YlvpDLix3S-U8fd-gqRwPcWXAXm8JwjL'
output = 'data.zip'
gdown.download(url, output, quiet=True)
gdown.extractall('data.zip')

In [ ]:
def load_vid(path:str) ->List[float]:
    cap=cv2.VideoCapture(path)
    frames=[]
    for i in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))):
        ret,frame=cap.read()
        frame=tf.image.rgb_to_grayscale(frame)
        frames.append(frame[190:236,80:220,:])
    cap.release()
    
    mean=tf.math.reduce_mean(frames)
    std=tf.math.reduce_std(tf.cast(frames, tf.float32))
    return tf.cast((frames-mean),tf.float32)/std

In [ ]:
vocab=[]
for x in " abcdefghijklmnopqrstuvwxyz'1234567890?!":
    vocab.append(x)


In [ ]:
char_to_num = tf.keras.layers.StringLookup(vocabulary=vocab, oov_token="")
num_to_char = tf.keras.layers.StringLookup(
    vocabulary=char_to_num.get_vocabulary(), oov_token="", invert=True
)

In [ ]:
def load_alignments(path:str) -> List[str]: 
    with open(path, 'r') as f: 
        lines = f.readlines() 
    tokens = []
    for line in lines:
        line = line.split()
        if line[2] != 'sil': 
            tokens = [*tokens,' ',line[2]]
    return char_to_num(tf.reshape(tf.strings.unicode_split(tokens, input_encoding='UTF-8'), (-1)))[1:]

In [ ]:
def load_data(path: str):
    path = path.numpy().decode('utf-8')
    file_name = path.split('/')[-1].split('.')[0]
    # File name splitting for windows
#file_name = path.split('\\')[-1].split('.')[0]
    video_path = os.path.join('data','s1',f'{file_name}.mpg')
    alignment_path = os.path.join('data','alignments','s1',f'{file_name}.align')
    frames = load_vid(video_path) 
    alignments = load_alignments(alignment_path)
    
    return frames, alignments

In [ ]:
def mappable_function(path:str):
    result = tf.py_function(load_data, [path], (tf.float32, tf.int64))
    return result

In [ ]:
data = tf.data.Dataset.list_files('./data/s1/*.mpg')

In [ ]:
data = data.shuffle(500)
data = data.map(mappable_function)
type(data)

In [ ]:
data = data.padded_batch(2, padded_shapes=([75,None,None,None],[40]))
data = data.prefetch(tf.data.AUTOTUNE)


In [ ]:
train=data.take(int(len(data)*0.9))
test2=data.skip(int(len(data)*0.9)).take(int(len(data)*0.1))

In [ ]:
sampl=train.as_numpy_iterator().next()

In [ ]:
sampl

In [ ]:
test3=list(test2)

In [ ]:
test3[0][0]

In [ ]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Conv3D, LSTM, Dense, Dropout, Bidirectional, MaxPool3D, Activation, Reshape, SpatialDropout3D, BatchNormalization, TimeDistributed, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler

In [ ]:
# with tpu_strategy.scope(): 
model = Sequential()
model.add(Conv3D(128, 3, input_shape=(75,46,140,1), padding='same'))
model.add(Activation('relu'))
model.add(MaxPool3D((1,2,2)))

model.add(Conv3D(256, 3, padding='same'))
model.add(Activation('relu'))
model.add(MaxPool3D((1,2,2)))

model.add(Conv3D(75, 3, padding='same'))
model.add(Activation('relu'))
model.add(MaxPool3D((1,2,2)))

model.add(TimeDistributed(Flatten()))

model.add(Bidirectional(LSTM(256, kernel_initializer='Orthogonal', return_sequences=True)))
model.add(Dropout(.5))

model.add(Bidirectional(LSTM(256, kernel_initializer='Orthogonal', return_sequences=True)))
model.add(Dropout(.5))

model.add(Dense(char_to_num.vocabulary_size(), kernel_initializer='he_normal', activation='softmax'))

In [ ]:
def CTCLoss(y_true, y_pred):
    batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
    input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
    label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

    input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
    label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")

    loss = tf.keras.backend.ctc_batch_cost(y_true, y_pred, input_length, label_length)
    return loss

In [ ]:
model.load_model('/kaggle/input/moedl1/lipnet.h5')

In [ ]:
graph=tf.function(CTCLoss)

In [ ]:
initial_learning_rate = 0.1
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.96,
    staircase=True)

In [ ]:
model.compile(
  optimizer=tf.keras.optimizers.Adadelta(learning_rate=lr_schedule),
  loss=CTCLoss
    
  )

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5,verbose=1)

In [ ]:
hist=model.fit(train, 
    epochs=90, 
    validation_data=test2,
    callbacks=[early_stopping]
)

In [ ]:
model2=tf.keras.saving.load_model('/kaggle/input/moedl1/lipnet.h5',custom_objects='CTCloss')

In [ ]:
model2 = tf.keras.saving.load_model('/kaggle/input/moedl1/lipnet.h5', custom_objects={'CTCLoss': CTCLoss})

In [ ]:
yhat=model2.predict(sampl[0])

In [ ]:
yhat.shape

In [ ]:
sequence_length = [75, 75]

In [ ]:
yhat = tf.transpose(yhat, perm=[1, 0, 2])

In [ ]:
tf.strings.reduce_join([num_to_char(y) for y in [tf.argmax(x) for x in yhat]])

In [ ]:
decoded=tf.nn.ctc_beam_search_decoder(
    yhat,sequence_length, beam_width=1, top_paths=1
)[0][0]


In [ ]:
decoded=tf.nn.ctc_beam_search_decoder(
    yhat,sequence_length, beam_width=1, top_paths=1
)[0]
decoded
#sparse tensor ke phle ek bracket hai it means ki  its a not a tensor but a list 
#to convert it to a tensor apan ko list ke us element ko access karna hoga

In [ ]:
decoded=tf.nn.ctc_beam_search_decoder(
    yhat,sequence_length, beam_width=1, top_paths=1
)
decoded

In [ ]:
decoded


In [ ]:
dense_tensor = tf.sparse.to_dense(decoded)


In [ ]:
dense_tensor

In [ ]:
# Apologies for the confusion in my previous response. The error message suggests that the length of the sequence_length argument provided to the tf.nn.ctc_beam_search_decoder function does not match the batch_size.

# In your case, it seems like you have mistakenly swapped the dimensions of your yhat tensor. The shape of yhat should be (75, 2, 41) instead of (2, 75, 41). The first dimension corresponds to the time steps, the second dimension represents the batch size, and the last dimension represents the number of classes.

# To fix the issue, you need to transpose the dimensions of your yhat tensor before passing it to the tf.nn.ctc_beam_search_decoder function. Here's an updated code snippet:
#the reason why i was getting the error

In [ ]:
[tf.strings.reduce_join([num_to_char(word) for word in sentence]) for sentence in dense_tensor]

In [ ]:
decoded2 = tf.keras.backend.ctc_decode(yhat, input_length=[75,75], greedy=True)[0][0].numpy()

In [ ]:
[tf.strings.reduce_join([num_to_char(word) for word in sentence]) for sentence in decoded2]

In [ ]:
[tf.strings.reduce_join([num_to_char(word) for word in sentence]) for sentence in sampl[1]]

In [ ]:
def output(path: str):
    path=tf.convert_to_tensor(path)
    data=load_data(path)
    exp=tf.expand_dims(samp[0],axis=0)
    out=model2.predict(exp)
    out=tf.transpose(out, perm=[1, 0, 2])
    dec=tf.nn.ctc_beam_search_decoder(
    out,sequence_length=[75], beam_width=1, top_paths=1)[0][0]
    den=tf.sparse.to_dense(dec)
    liz=[tf.strings.reduce_join([num_to_char(word) for word in sentence]) for sentence in den]
    return liz

In [ ]:
output('/kaggle/working/data/s1/sgio8p.mpg')


In [ ]:
# np.expand_dims and tf.expand dims basically adds another dimension in row/column
# since model me 1st dimension batch ka hota hai and agar wo dimension missing ho to 
#model error de deta hai
samp[0]

In [ ]:
tf.expand_dims(samp[0],axis=0)

In [ ]:
sampo=load_vid('/kaggle/input/ptaniyaar/video_2023-06-23_23-35-55.mp4')
# exp=tf.expand_dims(sampo[0],axis=0)
# out=model2.predict(exp)
# out=tf.transpose(out, perm=[1, 0, 2])
# dec=tf.nn.ctc_beam_search_decoder(
# out,sequence_length=[104], beam_width=1, top_paths=1)[0][0]
# den=tf.sparse.to_dense(dec)
# liz=[tf.strings.reduce_join([num_to_char(word) for word in sentence]) for sentence in den]


In [ ]:
sampo

In [ ]:
type(tesr)

In [ ]:
tesr

In [ ]:
type(tesr)
tesr

In [ ]:
sampo_reshaped = tf.reshape(sampo, (1,44, 46, 140, 1))

In [ ]:
tesr=tf.data.Dataset.from_tensor_slices(sampo_reshaped)

In [ ]:
tesr=tesr.padded_batch(1,padded_shapes=([75,None,None,None]))

In [ ]:
tesr

In [ ]:

out=model2.predict(tesr)

In [ ]:
sampo

In [ ]:
type(sampo)

In [ ]:
# out=tf.transpose(out, perm=[1, 0, 2])
dec = tf.keras.backend.ctc_decode(out, input_length=[75], greedy=True)[0][0].numpy()
# den=tf.sparse.to_dense(dec)
liz=[tf.strings.reduce_join([num_to_char(word) for word in sentence]) for sentence in den]

In [ ]:
liz

In [ ]:
out.shape